In [1]:
from info_nas.datasets.networks.pretrained import pretrain_network_cifar
from info_nas.datasets.networks.utils import load_nasbench
from nasbench_pytorch.datasets.cifar10 import prepare_dataset

In [2]:
seed = 42
batch_size = 32
num_workers = 8

dataset = prepare_dataset(batch_size, root='../data/', validation_size=1000, random_state=seed,
                          num_workers=num_workers)
train, n_train, val, n_val, test, n_test = dataset

--- Preparing CIFAR10 Data ---
Files already downloaded and verified
Files already downloaded and verified
--- CIFAR10 Data Prepared ---


In [3]:
import os

os.listdir('checkpoints')

['00005c142e6f48ac74fdcf73e3439874.tar',
 '0000718c96730a92ea6d5242f8ce77e2.tar',
 '0000cb5372fad9d62c470df699ac6d52.tar',
 '0000fa05697179112aaf69c2f0a51a0f.tar',
 '00010e19334eb9dcbe1fe61a98d524c2.tar',
 '00011986aeb27419a476d219f3d9ba14.tar',
 '000170f80dad8e2ef0b85c11c483eec1.tar',
 '00017a789793977dbadcf19592449804.tar',
 '000191735c964ed3522167de6c847cfe.tar',
 '00019e2cbab0bc93ca1a7816a099b2bd.tar',
 '0001a2f6c8977346ccd12fa0c435bf42.tar',
 '0001b322e18b86665c067ffa09e46897.tar',
 '000210d80242349a644f8872696c7f5c.tar',
 '000262be11e87946a203f65ebfbab64a.tar']

In [4]:
from nasbench import api

nasbench_path = '/home/gabi/diplomka/nasbench/nasbench_only108.tfrecord'
nb = api.NASBench(nasbench_path)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 33 seconds


In [6]:
import torch
from info_nas.datasets.networks.utils import load_trained_net

device = torch.device('cuda')

net_path = 'checkpoints/0000718c96730a92ea6d5242f8ce77e2.tar'
net, info = load_trained_net(net_path, nb)
net = net.to(device)
net

Network(
  (layers): ModuleList(
    (0): ConvBnRelu(
      (conv_bn_relu): Sequential(
        (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): Cell(
      (vertex_op): ModuleList(
        (0): None
        (1): Conv1x1BnRelu(
          (conv1x1): ConvBnRelu(
            (conv_bn_relu): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
          )
        )
        (2): Conv1x1BnRelu(
          (conv1x1): ConvBnRelu(
            (conv_bn_relu): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [8]:
for batch_idx, (inputs, targets) in enumerate(val):
    inputs, targets = inputs.to(device), targets.to(device)
    break
    
print(inputs.shape)
print(targets.shape)

torch.Size([32, 3, 32, 32])
torch.Size([32])


In [9]:
with torch.no_grad():
    outputs = net(inputs)
    
outputs.shape

torch.Size([32, 10])

In [12]:
with torch.no_grad():
    out_list = net.get_cell_outputs(inputs, return_inputs=False)
    
[print(o.shape) for o in out_list]
print()

torch.Size([32, 128, 32, 32])
torch.Size([32, 128, 32, 32])
torch.Size([32, 128, 32, 32])
torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 16, 16])
torch.Size([32, 512, 8, 8])
torch.Size([32, 512, 8, 8])
torch.Size([32, 512, 8, 8])
torch.Size([32, 10])



In [15]:
with torch.no_grad():
    in_list, out_list = net.get_cell_outputs(inputs, return_inputs=True)
    
[print(i.shape, ' -> ', o.shape) for i, o in zip(in_list, out_list)]
print()

torch.Size([32, 128, 32, 32])  ->  torch.Size([32, 128, 32, 32])
torch.Size([32, 128, 32, 32])  ->  torch.Size([32, 128, 32, 32])
torch.Size([32, 128, 32, 32])  ->  torch.Size([32, 128, 32, 32])
torch.Size([32, 128, 16, 16])  ->  torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 16, 16])  ->  torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 16, 16])  ->  torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 8, 8])  ->  torch.Size([32, 512, 8, 8])
torch.Size([32, 512, 8, 8])  ->  torch.Size([32, 512, 8, 8])
torch.Size([32, 512, 8, 8])  ->  torch.Size([32, 512, 8, 8])
torch.Size([32, 512, 8, 8])  ->  torch.Size([32, 10])

